In [4]:
from x3d_revised import x3d_with_regression
import torch

# MODEL_PATH = '../model_pts/x3d_m_no_reg2.pt'
device = 'cuda'

model = x3d_with_regression(pretrained = True, hidden_dim= 9408, model_num_class = 5, head_activation = None, in_between= 256).to(device)
# model.load_state_dict(torch.load(MODEL_PATH), strict= False)
model.eval()

x3d_with_regression(
  (model): Net(
    (blocks): ModuleList(
      (0): ResNetBasicStem(
        (conv): Conv2plus1d(
          (conv_t): Conv3d(3, 24, kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), bias=False)
          (conv_xy): Conv3d(24, 24, kernel_size=(5, 1, 1), stride=(1, 1, 1), padding=(2, 0, 0), groups=24, bias=False)
        )
        (norm): BatchNorm3d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (1): ResStage(
        (res_blocks): ModuleList(
          (0): ResBlock(
            (branch1_conv): Conv3d(24, 24, kernel_size=(1, 1, 1), stride=(1, 2, 2), bias=False)
            (branch2): BottleneckBlock(
              (conv_a): Conv3d(24, 54, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
              (norm_a): BatchNorm3d(54, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (act_a): ReLU()
              (conv_b): Conv3d(54, 54, kernel_size=(3, 3, 3), stride=(

In [5]:
import cProfile
input = torch.zeros(size=(1, 3, 13, 182, 182), device= device)
with torch.no_grad():
    cProfile.run('for i in range(100): \n    model(input)')

         386103 function calls (339303 primitive calls) in 2.376 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    2.376    2.376 <string>:1(<module>)
      100    0.000    0.000    0.000    0.000 _VF.py:26(__getattr__)
     8300    0.002    0.000    0.004    0.000 __init__.py:31(__get__)
     7000    0.008    0.000    0.183    0.000 activation.py:101(forward)
     1500    0.001    0.000    0.032    0.000 activation.py:293(forward)
     8300    0.040    0.000    0.454    0.000 batchnorm.py:137(forward)
     8300    0.007    0.000    0.012    0.000 batchnorm.py:514(_check_input_dim)
     4100    0.004    0.000    0.006    0.000 container.py:194(__iter__)
4100/2600    0.028    0.000    0.736    0.000 container.py:202(forward)
      500    0.002    0.000    0.003    0.000 container.py:268(_get_abs_string_index)
      500    0.001    0.000    0.005    0.000 container.py:277(__getitem__)
     1100   

In [6]:
from thop import profile

flops, params = profile(model, inputs=(input, ))

print('FLOPs = ' + str(flops/1000**3) + 'G')
print('Params = ' + str(params/1000**2) + 'M')

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv3d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm3d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_avgpool() for <class 'torch.nn.modules.pooling.AvgPool3d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool3d'>.
FLOPs = 2.907538829G
Params = 2.984919M
